# Competiciones de Kaggle
En el sitio web de Kaggle, podremos encontrar [competiciones](https://www.kaggle.com/competitions), mediante las cuales se fomenta la práctica de las distintas habilidades de Data Science, suministrando grandes dataset de manera pública y alentando a los usuarios a realizar sus propias mediciones/predicciones y compartirlas, obteniendo calificación por sus resultados y compitiendo así con el resto de los participantes.

NOTA: yo ya hice las *submission* correspondientes a las prácticas durante el curso, pero ahora las haré de nuevo, de manera de poder acostumbrarme a cuál sería la dinámica de hacerlo por mi cuenta.

In [50]:
print('Iniciando...')
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

iowa_houses_filepath = 'assets/input/house_competition/train.csv'
iowa_data = pd.read_csv(iowa_houses_filepath)

print('Datos cargados')

Iniciando...
Datos cargados


In [58]:
# primero inspecciono los datos por arriba
# iowa_data.head()

In [59]:
# iowa_data.describe()

**OJO:** En primera instancia había decidido hacer un *dropna()* para eliminar filas que tuvieran valores nulos, pero resultaba en que me quedaba un dataset vacío. Esto creo que se debe a que el dataset contiene columnas que están completamente formadas por valores nulos (por tanto se vaciaba por completo), esto lo noté al hacer un *describe()* y ver que posee 38 columnas de las 80 que contiene la tabla original, esto asumo que es porque esta función debe filtrar las columnas nulas directamente.

Continúo con la construcción del modelo, respecto a la *features* comienzo eligiéndolas por intuición, para luego ir probando si hay mejoras al tomar otras, aunque se supone que más adelante veremos técnicas para seleccionarlas.

In [61]:
y = iowa_data.SalePrice

# Comienzo eligiendo por intuición (ojo, no 'nan' ni valores no numéricos)
cand_features = [
    ['MSSubClass', 'LotArea', '1stFlrSF','FullBath', 'OverallQual', 'OverallCond', 'YearBuilt'],
    ['MSSubClass', 'LotArea', '1stFlrSF','FullBath', 'OverallQual', 'OverallCond', 'YearBuilt', 'BedroomAbvGr'],
    ['MSSubClass', 'LotArea', '1stFlrSF','FullBath', 'OverallQual', 'OverallCond', 'YearBuilt','BedroomAbvGr', 'KitchenAbvGr' ],
]
features = cand_features[2]
X = iowa_data[features]

**NOTA:** ya observo una primera complicación a la hora de elegir las *features* donde, si elijo alguna que posee valores no numéricos (ejemplo, MSZoning toma valores como 'RL', 'A'), resultará en un error ya que el modelo no puede resolverlo (se verá más adelante como solucionar esto). Por tanto, a priori, debo seleccionar solo aquellas que sean numéricas.

No había notado, pero en el curso te mostraban cuáles columnas no te darían conflictos:
- 'MSSubClass'
- 'LotArea'
- 'OverallQual'
- 'OverallCond'
- 'YearBuilt'
- 'YearRemodAdd'
- '1stFlrSF'
- '2ndFlrSF'
- 'LowQualFinSF'
- 'GrLivArea'
- 'FullBath'
- 'HalfBath'
- 'BedroomAbvGr'
- 'KitchenAbvGr'
- 'TotRmsAbvGrd'
- 'Fireplaces'
- 'WoodDeckSF'
- 'OpenPorchSF'
- 'EnclosedPorch'
- '3SsnPorch'
- 'ScreenPorch'
- 'PoolArea'
- 'MiscVal'
- 'MoSold'
- 'YrSold'

In [62]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

model = RandomForestRegressor(random_state=1)
model.fit(train_X,train_y)
val_predictions = model.predict(val_X)
val_mae = mean_absolute_error(val_y, val_predictions)

print('Validando el modelo obtengo un MAE: {:,.0f}'.format(val_mae))

Validando el modelo obtengo un MAE: 21,248


Suponiendo que ya valide mi modelo (deberia hacer pruebas para elegir cuál es el mejor), ya puedo entrenarlo con la data total (train + validation) y así obtener el modelo con la mayor precisión posible listo para realizar las predicciones sobre la data de la competencia (test.csv) y luego subir los resultados mediante la *submission* (debo generar un archivo csv con las predicciones).

In [63]:
# Modelo final entrenado con la data total
model.fit(X, y)

# Predicciones sobre la data 'test' de la competencia
test_home_filepath = 'assets/input/house_competition/test.csv'
test_home_data = pd.read_csv(test_home_filepath)
test_X = test_home_data[features]
test_predictions = model.predict(test_X)

# genero csv
output = pd.DataFrame({'Id': test_home_data.Id, 'SalePrice': test_predictions})
output.to_csv('assets/output/home_submission.csv', index=False)